In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.signal import find_peaks
from pandas import read_excel
from pandas import read_csv
from time import time 
import pickle
from obspy.signal.detrend import polynomial
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AR
from xgboost import XGBRegressor 

# Dummies

In [2]:
def dum_fest(df):
    cata = []; fes = read_csv("Excels/festivos.csv")
    for j in range(len(fes.index)):
            festiveDay = df.loc[(df.year==fes["Year"][j]) & (df.month==fes["Month"][j]) & (df.day==fes["Day"][j])].index.values.tolist()
            cata = cata + festiveDay

    df['fest'] = np.nan; df.fest = df.fest.fillna(pd.Series(-1, index=cata))
    df.fest = df.fest.fillna(0)
#     df = df.query("fest == 0")
#     df = df.drop("fest", axis = 1)
    return df

In [3]:
# lunes   martes-miercoles-jueves   viernes   sabado   domingo
def dum_dia(df):
    df.día_semana=df.día_semana.apply(lambda x: 1 if x==1 else (2 if x in [2,3,4] else (3 if x==5 else (4 if x==6 else 0))))
    df = pd.get_dummies(df, prefix=['d_sm'], columns=['día_semana'])
    df = df.drop("d_sm_0", axis = 1)
    return df

In [4]:
def dum_seas(df):
    df['seas'] = np.nan
    pr = df.query('day == 21 and month ==3').index
    vr = df.query('day == 21 and month == 6').index
    ot = df.query('day == 21 and month == 9').index
    inv= df.query('day == 21 and month == 12').index

    df.seas = df.seas.fillna(pd.Series(0, index=pr))
    df.seas = df.seas.fillna(pd.Series(1, index=vr))
    df.seas = df.seas.fillna(pd.Series(2, index=ot))
    df.seas = df.seas.fillna(pd.Series(3, index=inv))

    df.seas = df.seas.fillna(method= 'ffill')
    df.seas = df.seas.fillna(method= 'bfill')

    df = pd.get_dummies(df, prefix=['s_s'], columns=['seas'])
    df = df.drop("s_s_3.0", axis = 1)

    return df

In [5]:
#     Enero, Febrero, Marzo, Abril, Mayo, Junio, Julio, Agosto, Septiembre, Octubre, Diciembre
def dum_mensual(df):
    df = pd.get_dummies(df, prefix=['mon'], columns=['month'])
    df = df.drop("mon_12", axis = 1)
    return df

In [6]:
def fm1(i):
    df = pd.read_pickle("dflimpios.pkl")
    if i==0:
        df = dum_fest(df)
    if i==1:
        df = dum_dia(df)
    if i==2:
        df = dum_seas(df)
    if i==3:
        df = dum_mensual(df)
    if i==4:
        df = dum_mensual(dum_seas(dum_dia(dum_fest(df)))) 
    return df
fm1(1)

In [7]:
def sonas(): 
    return ["zona_{}".format(i+1) for i in range(26)]

# Funciones para regresión

In [8]:
def get_zon_hor(h,z):
    hora_n = df.query('hora == {}'.format(h))
    zon_n = hora_n['{}'.format(z)]
#     zon_n = polynomial(zon_n, order=2)
    return zon_n.values

In [9]:
def get_dums(h):
    dem = ['hora',]# 'year', 'month']
    zro =  sonas() #+dem
    sin_zonas = df[df["hora"]==h]
    sin_zonas=sin_zonas.reset_index()
    sin_zonas = sin_zonas.drop(zro,  axis=1)
    sin_zonas = sin_zonas.drop('index', axis=1)
    return  sin_zonas

In [10]:
def get_max_pts(h, z, armon):
    from  scipy import signal
    x = get_zon_hor(1, "zona_1")
    f, asd = signal.periodogram(x, fs=1)
    peaks, _ = find_peaks(asd, height=0)
    altura = _['peak_heights']
    fan = 1/(f[peaks])
    app = pd.DataFrame(altura, fan, columns=["altura_pk"]).sort_values(by="altura_pk",ascending=False).head(armon).iloc[armon-1:]
    return app;

In [11]:
def get_sc_coef(h, z, armon):
    X = pd.DataFrame()
    t = np.arange(len(df[df["hora"]==h]))
    X["te"] = t 
    p = get_max_pts(h, z, armon)
    for i  in p.index:
        X["{}_sen".format(i)] = (np.sin((t*2*np.pi)/i))
        X["{}_cos".format(i)] = (np.cos((t*2*np.pi)/i))
    X["unos"] = 1
    X = X.drop("te", axis = 1)
    return X

In [12]:
def kron_conv(h, z, armon):
    dm1 = get_dums(h); dm2 = get_sc_coef(h, z, armon)
    d3 = pd.DataFrame(); kron = pd.DataFrame()
    for i in dm2.columns:
        d3 = dm1.multiply(dm2[f"{i}"], axis="index")
        d3.columns = [f"{j}_{i}" for j in dm1.columns]
        kron = pd.concat([kron, d3], axis = 1)
    X = kron.reset_index()
    X2 = pd.concat([X, dm1, dm2], sort = False, axis=1)
    X2 = X2.drop('index', axis = 1)
    X2 = X2.drop('unos', axis = 1)
    return X2

In [13]:
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ma = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return ma

In [14]:
def get_train_split_sick(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/len(y), shuffle = False)
    return X_train, X_test, y_train, y_test

In [15]:
def get_train_split_xgb(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 1/len(y_test), shuffle = False)
    return X_train, X_test, X_val, y_train, y_test, y_val 

# Llamar datos

In [3]:
pkl_file = open('Excels/mapes_1dia.pkl', 'rb')
EAD = pickle.load(pkl_file)
pkl_file.close()
ED = EAD["mape"]

## Guardar datos

In [ ]:
beg = time()
for z in range(1, 27):
    EM = pd.DataFrame()
    EM[["model", "armon", "dli"]] = ED[f"zona_{z}"].idxmin(axis = 1).str.split('_', expand=True)
    EM[["armon", "dli"]] = EM[["armon", "dli"]].astype(int, errors = 'ignore')
    EM = EM.replace(['Sck', 'XGB','AR'],[1,2, 3])

    for h, a, armon, datos in zip(EM.index, EM["model"], EM["armon"], EM["dli"]):  
        
        if armon and datos == None:
            y = np.array(get_zon_hor(h,f"zona_{z}"))
            y_train, y_test = train_test_split(y, test_size = 1/len(y), shuffle = False)
            regresar = AR(y_train).fit()
            # Guradar modelo regresar
            output = open(f'Modelos/Zona{z}_hora{h}_AR.pkl', 'wb')
            pickle.dump(regresar, output)
            output.close()
            
        if a == 1:
            df = fm1(datos)
            y = np.array(get_zon_hor(h,f"zona_{z}"))
            X = kron_conv(h,f"zona_{z}", armon)
            X_tr, X_te, y_tr, y_te = get_train_split_sick(X, y)
            regress = LinearRegression(n_jobs = -1).fit(X_tr, y_tr)
            # Guardar modelo regress
            output = open(f'Modelos/Zona{z}_hora{h}.pkl', 'wb')
            pickle.dump(regress, output)
            output.close()
            
        if a == 2:
            df = fm1(datos)
            y = np.array(get_zon_hor(h,f"zona_{z}"))
            X = kron_conv(h,f"zona_{z}", armon)
            X_train, X_test, X_val, y_train, y_test, y_val = get_train_split_xgb(X, y)
            regress2 = XGBRegressor(objective ='reg:squarederror', n_jobs=-1, max_depth=3, learning_rate=1.5, n_estimators=5)
            regress2.fit(X_train, y_train, eval_set=[(X,y),(X_val,y_val)], verbose=0 , early_stopping_rounds=50)
            #Guardar modelo regress2
            output = open(f'Modelos/Zona{z}_hora{h}.pkl', 'wb')
            pickle.dump(regress2, output)
            output.close()
end = time()
end - beg

In [ ]:
regress = {}
for z in range(1, 27):
    EM = pd.DataFrame()
    EM[["model", "armon", "dli"]] = ED[f"zona_{z}"].idxmin(axis = 1).str.split('_', expand=True)
    EM[["armon", "dli"]] = EM[["armon", "dli"]].astype(int, errors = 'ignore')
    EM = EM.replace(['Sck', 'XGB','AR'],[1,2, 3])

    for h, a, armon, datos in zip(EM.index, EM["model"], EM["armon"], EM["dli"]):  
        
        if armon and datos == None:
            y = np.array(get_zon_hor(h,f"zona_{z}"))
            y_train, y_test = train_test_split(y, test_size = 1/len(y), shuffle = False)
            regress[f"zona_{z}_hora_{h}_arm_{armon}_dll_{datos}_AR"] = AR(y_train).fit()
            # Guradar modelo regresar
            
        if a == 1:
            df = fm1(datos)
            y = np.array(get_zon_hor(h,f"zona_{z}"))
            X = kron_conv(h,f"zona_{z}", armon)
            X_tr, X_te, y_tr, y_te = get_train_split_sick(X, y)
            regress[f"zona_{z}_hora_{h}_arm_{armon}_dll_{datos}_sck"] = LinearRegression(n_jobs = -1).fit(X_tr, y_tr)
            # Guardar modelo regress
            
        if a == 2:
            df = fm1(datos)
            y = np.array(get_zon_hor(h,f"zona_{z}"))
            X = kron_conv(h, f"zona_{z}", armon)
            X_train, X_test, X_val, y_train, y_test, y_val = get_train_split_xgb(X, y)
            regress[f"zona_{z}_hora_{h}_arm_{armon}_dll_{datos}_XGB"] = XGBRegressor(objective ='reg:squarederror', n_jobs=-1, max_depth=3, learning_rate=1.5, n_estimators=5)
            regress[f"zona_{z}_hora_{h}_arm_{armon}_dll_{datos}_XGB"].fit(X_train, y_train, eval_set=[(X,y),(X_val,y_val)], verbose=0 , early_stopping_rounds=50)
            #Guardar modelo regress
            
output = open(f'Modelos/modelos.pkl', 'wb')
pickle.dump(regress, output)
output.close()